In [1]:
import json
import glob
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('precision', 1)

In [2]:
# !pip install pandas==1.0.3

In [3]:
# Path to the JSON files
# directory_path_account = '/home/izi-99/demo/data/accounts/'
# directory_path_cc = '/home/izi-99/demo/cards/'
# directory_path_saving = '/home/izi-99/demo/savings_accounts/'

In [4]:
#For all json

def set_path(directory_path):
    
    #Adding all the file names; 
    file_path = []
    for file in glob.glob(directory_path):
        file_path.append(file)
        
    return file_path

def set_file(file_path):

    #Loading all the JSON files using file names
    json_file = []
    for ind in range(len(file_path)):
        with open(file_path[ind]) as f:
            json_file.append(json.load(f))
            
    return json_file

def convert_file(json_file):

    #Converting the JSON files to Data Frames
    df = pd.DataFrame()
    for ind in range(len(json_file)):
        df = df.append(pd.json_normalize(json_file[ind]))
    
    return df

def transform_nan(df):
    if df == "nan":
        return None
    else:
        return df

def change_date(df):
    if str(df) == '0000-00-00 00:00:00' or str(df) == '0000-00-00 00:00:00.000':
        return pd.NaT
    else:
        return df

def transform_account(directory_path):  
    file_path = set_path(directory_path)
    json_file = set_file(file_path)
    data = convert_file(json_file)
    data = data.applymap(transform_nan)
#     data = data.fillna(0)
    data['ts'] =  pd.to_datetime(data['ts'],unit='ms')
    
    return data

In [5]:
df_account = transform_account('/home/izi-99/demo/data/accounts/*.json')
df_cc = transform_account('/home/izi-99/demo/data/cards/*.json')
df_savings = transform_account('/home/izi-99/demo/data/savings_accounts/*.json')

In [6]:
df_account = df_account.sort_values(by='ts')
df_cc = df_cc.sort_values(by='ts')
df_savings = df_savings.sort_values(by='ts')

In [7]:
df_account = df_account.reset_index(drop = True)
df_cc = df_cc.reset_index(drop = True)
df_savings = df_savings.reset_index(drop = True)

In [8]:
# use the Python pivot function to denormalize
df_account = df_account.pivot_table(index='ts', values=['set.card_id','data.account_id','set.savings_account_id'], columns='id', aggfunc = 'first')

In [9]:
# use the Python pivot function to denormalize
df_cc = df_cc.pivot_table(index='ts', values=['data.card_id','set.credit_used','data.monthly_limit'], columns='id', aggfunc = 'first')

In [10]:
# use the Python pivot function to denormalize
df_savings = df_savings.pivot_table(index='ts', values=['data.savings_account_id','set.balance'], columns='id', aggfunc = 'first')

In [11]:
df_merge1 = pd.merge(df_account,df_cc,on='ts',how='outer')
df_merge2 = pd.merge(df_merge1,df_savings, on='ts',how='outer')
df_merge2 = df_merge2.fillna(0).sort_values(by='ts')

In [12]:
df_merge2

data.account_id set.card_id set.savings_account_id data.card_id            data.monthly_limit            set.credit_used            data.savings_account_id set.balance
id                       a1globalid  a1globalid             a1globalid   c1globalid c2globalid         c1globalid c2globalid      c1globalid c2globalid             sa1globalid sa1globalid
ts                                                                                                                                                                                         
2020-01-01 07:30:00              a1           0                      0            0          0                0.0        0.0             0.0        0.0                       0         0.0
2020-01-01 15:00:00               0           0                    sa1            0          0                0.0        0.0             0.0        0.0                     sa1         0.0
2020-01-02 01:00:00               0          c1                      0           c1          0            30000.0        0.0             0.0        0.0                       0         0.0
2020-01-02 09:00:00               0           0                      0            0          0                0.0        0.0             0.0        0.0                       0     15000.0
2020-01-06 12:30:00               0           0                      0            0          0                0.0        0.0         12000.0        0.0                       0         0.0
2020-01-07 18:00:00               0           0                      0            0          0                0.0        0.0         19000.0        0.0                       0         0.0
2020-01-10 09:30:00               0           0                      0            0          0                0.0        0.0             0.0        0.0                       0     40000.0
2020-01-10 11:00:00               0           0                      0            0          0                0.0        0.0             0.0        0.0                       0     21000.0
2020-01-15 09:01:00               0                                  0            0          0                0.0        0.0             0.0        0.0                       0         0.0
2020-01-16 08:30:00               0          c2                      0            0         c2                0.0    70000.0             0.0        0.0                       0         0.0
2020-01-18 15:30:00               0           0                      0            0          0                0.0        0.0             0.0    37000.0                       0         0.0
2020-01-20 07:30:00               0           0                      0            0          0                0.0        0.0             0.0        0.0                       0     33000.0